# Data Cleaning

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope

In [4]:
df = pd.read_csv("../data/sba_national.csv", low_memory=False)
print(df.shape)
df.sample(3)

(899164, 27)


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
167296,2201954004,ARMANDO & ZULEMA Z. MARROQUIN,ALICE,TX,78332,TEXAS CERT. DEVEL COMPANY INC.,TX,443111,17-Jul-98,1998,...,N,N,NaN,14-Feb-01,"$221,000.00",$0.00,P I F,$0.00,"$239,000.00","$239,000.00"
646383,6468074008,A+ PAINTING SERVICES INC,MADISON,WI,53704,WELLS FARGO BANK NATL ASSOC,CA,238320,3-Jul-03,2003,...,Y,N,NaN,31-Aug-03,"$10,743.00",$0.00,P I F,$0.00,"$10,000.00","$5,000.00"
642803,6418384009,PC RELIANCE INC,COSTA MESA,CA,92627,BANK OF AMERICA NATL ASSOC,NC,423690,18-Jun-03,2003,...,Y,N,NaN,31-Dec-03,"$15,000.00",$0.00,P I F,$0.00,"$15,000.00","$7,500.00"


## Data Validation

In [5]:
df.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

### LoanNr_ChkDgt, Name, City, Zip, Bank, FranchiseCode, BalanceGross, ChgOffDate y ChgOffPrinGr

In [6]:
drop_cols = ["LoanNr_ChkDgt",   # tiene 100% de valores distintos
             "Name",            # tiene 86.7% de valores distintos
             "City",            # cuenta con 32581 ciudades diferentes
             "Zip",             # cuenta con 33611 codigos postales diferentes
             "Bank",            # cuenta con 5802 bancos diferentes
             "FranchiseCode",   # tiene 94.2% de codigos sin franquicia
             "BalanceGross",    # tiene practicamente 100% de valores en cero
             "ChgOffDate",      # fuga de informacion de la variable objetivo
             "ChgOffPrinGr"]    # fuga de informacion de la variable objetivo

In [7]:
df.drop(drop_cols, axis=1, inplace=True)
df.shape

(899164, 18)

### ApprovalDate, ApprovalFY y DisbursementDate

In [8]:
drop_cols = ["ApprovalDate",    # comportamiento similar a DisbursementDate
             "ApprovalFY"]      # comportamiento similar a DisbursementDate

In [9]:
# remplazamos las fechas de desembolsos nulas por las fechas de aprobacion
df["DisbursementDate"] = np.where(df["DisbursementDate"].isnull(), df["ApprovalDate"], df["DisbursementDate"])

In [10]:
# formateamos la fecha debido a que el año tiene solamente dos digitos
def format_date(app_date):
    date = app_date.split("-")
    year = date[2]
    if int(year) > 14: year = "19" + year
    else: year = "20" + year
    return f"{date[0]}-{date[1]}-{year}"

df["DisbursementDate"] = pd.to_datetime(df["DisbursementDate"].apply(format_date))

In [11]:
df.drop(drop_cols, axis=1, inplace=True)
df.shape

(899164, 16)

### State, BankState, Term, NoEmp, CreateJob, RetainedJob y UrbanRural

In [12]:
df["State"].unique()

array(['IN', 'OK', 'FL', 'CT', 'NJ', 'NC', 'IL', 'RI', 'TX', 'VA', 'TN',
       'AR', 'MN', 'MO', 'MA', 'CA', 'SC', 'LA', 'IA', 'OH', 'KY', 'MS',
       'NY', 'MD', 'PA', 'OR', 'ME', 'KS', 'MI', 'AK', 'WA', 'CO', 'MT',
       'WY', 'UT', 'NH', 'WV', 'ID', 'AZ', 'NV', 'WI', 'NM', 'GA', 'ND',
       'VT', 'AL', 'NE', 'SD', 'HI', 'DE', 'DC', nan], dtype=object)

In [13]:
df["BankState"].unique()

array(['OH', 'IN', 'OK', 'FL', 'DE', 'SD', 'AL', 'CT', 'GA', 'OR', 'MN',
       'RI', 'NC', 'TX', 'MD', 'NY', 'TN', 'SC', 'MS', 'MA', 'LA', 'IA',
       'VA', 'CA', 'IL', 'KY', 'PA', 'MO', 'WA', 'MI', 'UT', 'KS', 'WV',
       'WI', 'AZ', 'NJ', 'CO', 'ME', 'NH', 'AR', 'ND', 'MT', 'ID', nan,
       'WY', 'NM', 'DC', 'NV', 'NE', 'PR', 'HI', 'VT', 'AK', 'GU', 'AN',
       'EN', 'VI'], dtype=object)

In [14]:
df["Term"].value_counts()

Term
84     230162
60      89945
240     85982
120     77654
300     44727
        ...  
438         1
382         1
367         1
374         1
429         1
Name: count, Length: 412, dtype: int64

In [15]:
df["NoEmp"].value_counts()

NoEmp
1       154254
2       138297
3        90674
4        73644
5        60319
         ...  
414          1
604          1
1280         1
3089         1
3713         1
Name: count, Length: 599, dtype: int64

In [16]:
df["CreateJob"].value_counts()

CreateJob
0      629248
1       63174
2       57831
3       28806
4       20511
        ...  
171         1
146         1
214         1
179         1
860         1
Name: count, Length: 246, dtype: int64

In [17]:
df["RetainedJob"].value_counts()

RetainedJob
0      440403
1       88790
2       76851
3       49963
4       39666
        ...  
191         1
247         1
226         1
675         1
198         1
Name: count, Length: 358, dtype: int64

In [18]:
df["UrbanRural"].value_counts()

UrbanRural
1    470654
0    323167
2    105343
Name: count, dtype: int64

### DisbursementGross, GrAppv, SBA_Appv

In [19]:
# DisbursementGross: formateamos la variable de currency a entero
df["DisbursementGross"] = df["DisbursementGross"].str.replace(r"[\$,]", "", regex=True).astype(float)
df["DisbursementGross"] = df["DisbursementGross"].round().astype(int)
df["DisbursementGross"].value_counts()

DisbursementGross
50000      43787
100000     36714
25000      27387
150000     23373
10000      21328
           ...  
143118         1
23827          1
157338         1
60698          1
1086300        1
Name: count, Length: 118859, dtype: int64

In [20]:
# GrAppv: formateamos la variable de currency a entero
df["GrAppv"] = df["GrAppv"].str.replace(r"[\$,]", "", regex=True).astype(float)
df["GrAppv"] = df["GrAppv"].round().astype(int)
df["GrAppv"].value_counts()

GrAppv
50000      69394
25000      51258
100000     50977
10000      38366
150000     27624
           ...  
407542         1
220900         1
57587          1
531577         1
1086300        1
Name: count, Length: 22128, dtype: int64

In [21]:
# SBA_Appv: formateamos la variable de currency a entero
df["SBA_Appv"] = df["SBA_Appv"].str.replace(r"[\$,]", "", regex=True).astype(float)
df["SBA_Appv"] = df["SBA_Appv"].round().astype(int)
df["SBA_Appv"].value_counts()

SBA_Appv
25000     49579
12500     40147
5000      31135
50000     25047
10000     17009
          ...  
192949        1
212352        1
57072         1
32938         1
715674        1
Name: count, Length: 38326, dtype: int64

### NAICS

In [23]:
sector = {"11":"Agriculture, forestry, fishing and hunting",
          "21":"Mining, quarrying, and oil and gas extraction",
          "22":"Utilities",
          "23":"Construction",
          "31":"Manufacturing",
          "32":"Manufacturing",
          "33":"Manufacturing",
          "42":"Wholesale trade",
          "44":"Retail trade",
          "45":"Retail trade",
          "48":"Transportation and warehousing",
          "49":"Transportation and warehousing",
          "51":"Information",
          "52":"Finance and insurance",
          "53":"Real estate and rental and leasing",
          "54":"Professional, scientific, and technical services",
          "55":"Management of companies and enterprises",
          "56":"Administrative and support and waste management and remediation services",
          "61":"Educational services",
          "62":"Health care and social assistance",
          "71":"Arts, entertainment, and recreation",
          "72":"Accommodation and food services",
          "81":"Other services (except public administration)",
          "92":"Public administration",
          "0":"[Unallocated sector]"} # creamos la categoria "sector no asignado"}

In [24]:
# asignamos a NAICS el nombre del sector correspondiente
df["NAICS"] = df["NAICS"].astype(str).str[0:2]
df["NAICS"] = df["NAICS"].apply(lambda x: sector[x])
df["NAICS"]

0                              Retail trade
1           Accommodation and food services
2         Health care and social assistance
3                      [Unallocated sector]
4                      [Unallocated sector]
                        ...                
899159                         Retail trade
899160                         Retail trade
899161                        Manufacturing
899162                 [Unallocated sector]
899163                 [Unallocated sector]
Name: NAICS, Length: 899164, dtype: object

### NewExist, RevLineCr y LowDoc

In [25]:
# NewExist: convertimos a NaN los valores no definidos
df["NewExist"] = np.where(df["NewExist"]==0.0, np.nan, df["NewExist"])
df["NewExist"] = df["NewExist"].astype("Int64")
df["NewExist"].value_counts()

NewExist
1    644869
2    253125
Name: count, dtype: Int64

In [26]:
# RevLineCr: convertimos a enteros los valores definidos y a NaN los demas
# asumiremos que T (true) y 1 podrian ser Y (yes), y 0 podria ser N (no)
df["RevLineCr"] = np.where(df["RevLineCr"].isin(["Y","T"]), "1", df["RevLineCr"])
df["RevLineCr"] = np.where(df["RevLineCr"].isin(["N"]), "0", df["RevLineCr"])
df["RevLineCr"] = np.where(~df["RevLineCr"].isin(["1","0"]), np.nan, df["RevLineCr"])
df["RevLineCr"] = df["RevLineCr"].astype("Int64")
df["RevLineCr"].value_counts()

RevLineCr
0    677890
1    216704
Name: count, dtype: Int64

In [27]:
# LowDoc: convertimos a enteros los valores definidos y a NaN los demas
# asumiremos que 1 podria ser Y (yes), y que 0 podria ser N (no)
df["LowDoc"] = np.where(df["LowDoc"]=="Y", "1", df["LowDoc"])
df["LowDoc"] = np.where(df["LowDoc"]=="N", "0", df["LowDoc"])
df["LowDoc"] = np.where(~df["LowDoc"].isin(["1","0"]), np.nan, df["LowDoc"])
df["LowDoc"] = df["LowDoc"].astype("Int64")
df["LowDoc"].value_counts()

LowDoc
0    784313
1    110336
Name: count, dtype: Int64

### MIS_Status

In [28]:
# MIS_Status: convertimos los valores a enteros (1 = Default y 0 = Paid)
df["MIS_Status"] = np.where(df["MIS_Status"]=="CHGOFF", 1, df["MIS_Status"])
df["MIS_Status"] = np.where(df["MIS_Status"]=="P I F", 0, df["MIS_Status"])
df["MIS_Status"] = df["MIS_Status"].astype("Int64")
df["MIS_Status"].value_counts()

MIS_Status
0    739609
1    157558
Name: count, dtype: Int64

### **Summary**

In [29]:
# adicionalmente se visualizan y eliminan las instancias duplicadas
print(df[df.duplicated].shape)
display(df[df.duplicated].sample(3))

df.drop_duplicates(inplace=True)

(1829, 16)


,State,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,MIS_Status,GrAppv,SBA_Appv
61267,TX,TX,Health care and social assistance,306,5,1,0,0,0,0,0,1997-12-31,500000,0,500000,375000
308362,CA,CA,Transportation and warehousing,12,145,1,124,145,2,1,0,2008-01-31,1500,<NA>,1500,750
61219,CA,CA,Transportation and warehousing,12,82,2,82,0,2,0,0,2006-02-28,4000,0,4000,2000


In [30]:
print(df.shape)
df.dtypes

(897335, 16)


State                        object
BankState                    object
NAICS                        object
Term                          int64
NoEmp                         int64
NewExist                      Int64
CreateJob                     int64
RetainedJob                   int64
UrbanRural                    int64
RevLineCr                     Int64
LowDoc                        Int64
DisbursementDate     datetime64[ns]
DisbursementGross             int64
MIS_Status                    Int64
GrAppv                        int64
SBA_Appv                      int64
dtype: object

## Missing Values

In [31]:
df.isnull().sum()

State                  14
BankState            1562
NAICS                   0
Term                    0
NoEmp                   0
NewExist             1169
CreateJob               0
RetainedJob             0
UrbanRural              0
RevLineCr            4566
LowDoc               4512
DisbursementDate        0
DisbursementGross       0
MIS_Status           1951
GrAppv                  0
SBA_Appv                0
dtype: int64

### State y BankState

In [32]:
# imputamos los valores faltantes con la moda de la variable
mode = df["State"].mode()[0]
df["State"] = df["State"].fillna(mode)

In [33]:
# imputamos los valores faltantes con la moda de la variable
col_values = df[["BankState"]]
mode_imputer = SimpleImputer(strategy="most_frequent")
mode_imputer_fit = mode_imputer.fit(col_values)
df["BankState"] = mode_imputer_fit.transform(col_values)[:, 0]

### NewExist

In [34]:
# aplicamos imputacion multivariante de faltantes con KNN Imputer
cols_fit = df[["NewExist", "Term", "NoEmp", "CreateJob", "RetainedJob",
               "UrbanRural", "DisbursementGross", "GrAppv", "SBA_Appv"]]
k = int(round(np.sqrt(df.shape[0]), 2))
knn_imputer = KNNImputer(n_neighbors=k)
knn_imputer_fit = knn_imputer.fit(cols_fit)

cols_imputed = knn_imputer_fit.transform(cols_fit)
df["NewExist"] = cols_imputed[:, 0]

df["NewExist"] = round(df["NewExist"]).astype(int)
df["NewExist"].value_counts()

NewExist
1    644608
2    252727
Name: count, dtype: int64

### RevLineCr y LowDoc

In [36]:
# imputamos valores categoricos faltantes utilizando modelos de clasificacion
cols = ["RevLineCr", "LowDoc"]
for col in cols:
    df_ = df[[col, "Term", "NoEmp", "CreateJob", "RetainedJob",
              "UrbanRural", "DisbursementGross", "GrAppv", "SBA_Appv"]]

    df_train = df_.dropna(subset=[col])
    df_test = df_[df_[col].isnull()]

    X_train = df_train.drop(columns=[col])
    y_train = df_train[col]
    X_test = df_test.drop(columns=[col])

    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(X_train, y_train)

    y_test = rf_classifier.predict(X_test)
    df.loc[df[col].isnull(), col] = y_test

    df[col] = df[col].astype(int)
    print(f"{col}:")
    print(f"{df[col].value_counts()}\n")

RevLineCr:
RevLineCr
0    681358
1    215977
Name: count, dtype: int64

LowDoc:
LowDoc
0    787236
1    110099
Name: count, dtype: int64



### MIS_Status

In [37]:
# eliminamos todas las instancias nulas de la variable objetivo
df.dropna(subset=["MIS_Status"], inplace=True)
df["MIS_Status"] = df["MIS_Status"].astype(int)
df.shape

(895384, 16)

### **Summary**

In [38]:
print(df.shape)
df.isnull().sum()

(895384, 16)


State                0
BankState            0
NAICS                0
Term                 0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementDate     0
DisbursementGross    0
MIS_Status           0
GrAppv               0
SBA_Appv             0
dtype: int64

## Outlier Detection

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 895384 entries, 0 to 899163
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   State              895384 non-null  object        
 1   BankState          895384 non-null  object        
 2   NAICS              895384 non-null  object        
 3   Term               895384 non-null  int64         
 4   NoEmp              895384 non-null  int64         
 5   NewExist           895384 non-null  int64         
 6   CreateJob          895384 non-null  int64         
 7   RetainedJob        895384 non-null  int64         
 8   UrbanRural         895384 non-null  int64         
 9   RevLineCr          895384 non-null  int64         
 10  LowDoc             895384 non-null  int64         
 11  DisbursementDate   895384 non-null  datetime64[ns]
 12  DisbursementGross  895384 non-null  int64         
 13  MIS_Status         895384 non-null  int64        

### Term y NoEmp

In [40]:
# usamos la desviacion estandar y Z-score con umbral de 3 para detectar outliers
cols = ["Term", "NoEmp"]
for col in cols:
    media = df[col].mean()
    dest = df[col].std()
    umbral = 3

    z_scores = (df[col] - media) / dest
    df = df[(z_scores.abs() <= umbral)]

    print(f"{col}:")
    print(f"Media: {round(media, 1)}")
    print(f"Desviacion: {round(dest, 1)}")
    print(f"Limite Max: {round(media + (dest * umbral), 1)}")
    print(f"Outliers: {z_scores[z_scores > umbral].count()}\n")

print(df.shape)

Term:
Media: 110.9
Desviacion: 78.9
Limite Max: 347.7
Outliers: 183

NoEmp:
Media: 11.4
Desviacion: 73.9
Limite Max: 233.0
Outliers: 1444

(893757, 16)


### DisbursementGross

In [41]:
# usamos el rango intercuartilico IQR multiplicado con 7.5 como factor de escala
Q1 = df["DisbursementGross"][df["DisbursementGross"] > 0].quantile(0.25)
Q3 = df["DisbursementGross"][df["DisbursementGross"] > 0].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 7.5* IQR
outliers = df[(df["DisbursementGross"] < limite_inferior) | (df["DisbursementGross"] > limite_superior)].shape[0]

df = df[(df["DisbursementGross"] >= limite_inferior) & (df["DisbursementGross"] <= limite_superior)]

print(f"Rango Intercuartilico: {round(IQR, 1)}")
print(f"Limite Inferior: {round(limite_inferior, 1)}")
print(f"Limite Superior: {round(limite_superior, 1)}")
print(f"Outliers: {outliers}")
print(df.shape)

Rango Intercuartilico: 195500.0
Limite Inferior: -250750.0
Limite Superior: 1704250.0
Outliers: 4073
(889684, 16)


### GrAppv y SBA_Appv

In [42]:
# aplicamos el ajuste de una envolvente eliptica para datos multivariados
df_ = df[["GrAppv","SBA_Appv"]].copy()
# model_outlier = IsolationForest(contamination=0.01)
model_outlier = EllipticEnvelope(contamination=0.01)
model_outlier.fit(df_)

outliers = model_outlier.predict(df_)
df_["Outlier"] = np.where(outliers == -1, 1, 0)
df = df[df_["Outlier"] == 0]

out_count = df_[df_["Outlier"]==1].shape[0]
print(f"Outliers: {out_count}")
print(df.shape)

Outliers: 8897
(880787, 16)


### **Summary**

In [43]:
print(df.shape)
df.sample(3)

(880787, 16)


,State,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,MIS_Status,GrAppv,SBA_Appv
813805,WA,WA,Other services (except public administration),240,2,1,2,0,1,0,0,2005-07-13,256000,0,256000,256000
313763,MO,CA,Other services (except public administration),108,5,1,0,5,1,0,0,1999-09-30,310000,1,310000,232500
721664,NH,RI,"Professional, scientific, and technical services",84,1,1,0,1,1,1,0,2004-07-31,152993,0,35000,17500


In [44]:
df.to_csv("../data/sba_clean.csv", index=False)